In [3]:
"""
Import libraries
"""
import pandas as pd
import numpy as np
import seaborn as sns
import mlflow
import datetime as dt
import matplotlib.pyplot as plt

In [4]:
# File location
file = './data/yellow_tripdata_2022-01.parquet'
# Read in file
in_df = pd.read_parquet(file)
# Display
in_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
# Columns
in_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [6]:
# Calculate the trip duration in minutes
in_df['duration'] = (in_df['tpep_dropoff_datetime'] - in_df['tpep_pickup_datetime']).apply(lambda x: x.total_seconds()/60)
# Print head
in_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [7]:
# Calculate standard deviation of January trip durations
f'Standard Deviation of January Duration: {in_df["duration"].std():.4}'

'Standard Deviation of January Duration: 46.45'

In [8]:
# Remove the longer trips and save the df of inclusive values
sub_df = in_df[((in_df['duration'] >= 1) & (in_df['duration'] <= 60))]
# Now display the head
sub_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [9]:
# Print the percentage ratio of columns dropped
f'Post-Outlier Drop: {sub_df.shape[0]/in_df.shape[0]*100.0:.4f}%'

'Post-Outlier Drop: 98.2755%'

In [10]:
# Feature columns
feature_columns = ['PULocationID', 'DOLocationID']
label_columns = ["duration"]
# Create sub dataframe for the desired cols
feature_df = sub_df[feature_columns].astype('category')
# Now one hot encode them using the pandas helper method.
feature_df = pd.get_dummies(feature_df, columns=feature_columns)
# Now display head
feature_df.head()

,duration,PULocationID_1,PULocationID_2,PULocationID_3,PULocationID_4,PULocationID_5,PULocationID_6,PULocationID_7,PULocationID_8,PULocationID_9,...,DOLocationID_256,DOLocationID_257,DOLocationID_258,DOLocationID_259,DOLocationID_260,DOLocationID_261,DOLocationID_262,DOLocationID_263,DOLocationID_264,DOLocationID_265
0,17.816667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8.400000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8.966667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10.033333,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37.533333,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Calculate the number of columns
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421440 entries, 0 to 2463930
Columns: 516 entries, duration to DOLocationID_265
dtypes: category(1), uint8(515)
memory usage: 1.2 GB


In [12]:
# Create Dictionary Vectorizer if needed
from sklearn.feature_extraction import DictVectorizer
# Create the instance
dv = DictVectorizer()
# Pass in the dataframe and feed/transform.
result = dv.fit_transform()

In [13]:
# Train a simple linear regression model and calculate the rmse
# The model definition
from sklearn.linear_model import LinearRegression
# The metrics report for the item
from sklearn.metrics import mean_squared_error